In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Importing the dataset
data = pd.read_csv('amazon_alexa.tsv', delimiter = '\t', 
                      quoting = 3)

In [3]:
data.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [4]:
data.groupby('rating').describe()

feedback                                   
          count mean  std  min  25%  50%  75%  max
rating                                            
1         161.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2          96.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3         152.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
4         455.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
5        2286.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [5]:
data.shape

(3150, 5)

In [6]:
#Feature Engineering : New column to detect length odf reviw
data['len_review'] = data['verified_reviews'].apply(len)

In [7]:
data.head()

,rating,date,variation,verified_reviews,feedback,len_review
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,13
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,9
2,4,31-Jul-18,Walnut Finish,"""Sometimes while playing a game, you can answe...",1,197
3,5,31-Jul-18,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4...",1,174
4,5,31-Jul-18,Charcoal Fabric,Music,1,5


In [8]:
#Cleaning the texts
corpus = []
for i in range(0,3150):
    review = re.sub('[^a-zA-Z]', ' ', data['verified_reviews'][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

## BOW

In [9]:
#Creating a bag of words
cv = CountVectorizer(max_features=1500)

In [10]:
X = cv.fit_transform(corpus).toarray()
y = data.iloc[:,4].values

In [11]:
#Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [12]:
#Using Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr.predict(X_test)

from sklearn.metrics import (confusion_matrix,accuracy_score)

print(confusion_matrix(y_test,lr.predict(X_test)))
print('\n')
print('Accuracy for Logistic Regression: \n',accuracy_score(y_test, lr.predict(X_test)))

[[ 20  34]
 [  3 573]]


Accuracy for Logistic Regression: 
 0.94126984127


In [13]:
#Using XGBoost
import xgboost as xgb

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n')
print('Accuracy for XGBoost: \n',accuracy_score(y_test, classifier.predict(X_test)))

[[ 12  42]
 [  0 576]]


Accuracy for XGBoost: 
 0.933333333333


In [14]:
#Creating bag of words model
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(max_features=2000)
X_tf = tfidf.fit_transform(corpus)
y_tf = data.iloc[:,4].values

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, y_tf, test_size = 0.20, random_state = 0)

In [17]:
#Using Logistic Regression
from sklearn.linear_model import LogisticRegression
lr_tf = LogisticRegression()
lr_tf.fit(X_train,y_train)
lr_tf.predict(X_test)

from sklearn.metrics import (confusion_matrix,accuracy_score)

print(confusion_matrix(y_test,lr_tf.predict(X_test)))
print('\n')
print('Accuracy for Logistic Regression: \n',accuracy_score(y_test, lr_tf.predict(X_test)))

[[  2  52]
 [  0 576]]


Accuracy for Logistic Regression: 
 0.91746031746


In [18]:
vocab_cv = cv.vocabulary_
print(len(vocab_cv))

1500


In [19]:
vocab_tf = tfidf.vocabulary_
print(type(vocab_tf))

<class 'dict'>


In [20]:
cv_coeffs = lr.coef_
print(cv_coeffs.shape)

(1, 1500)


In [21]:
tf_coeffs = lr_tf.coef_
print(tf_coeffs.shape)

(1, 2000)


In [22]:
#Using XGBoost
import xgboost as xgb

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n')
print('Accuracy for XGBoost: \n',accuracy_score(y_test, classifier.predict(X_test)))

[[ 12  42]
 [  0 576]]


Accuracy for XGBoost: 
 0.933333333333


In [23]:
def get_feat_dict(feat_list):
    feat_dict={}
    idx=0
    for item in feat_list:
        feat_dict[item] = idx
        idx = idx+1
    return feat_dict    

In [24]:
cv_feat_dict = get_feat_dict(cv.get_feature_names())
tf_feat_dict = get_feat_dict(tfidf.get_feature_names())

## Sentiment Analysis

In [25]:
#Contextual sentiment for count vector
sentiList_cv = []
threshold = 0.5
for word, index in vocab_cv.items():
    weight = cv_coeffs[0][cv_feat_dict[word]]
    if weight > threshold or weight < - threshold:
        sentiList_cv.append((word,weight))

In [26]:
len(sentiList_cv)

168

In [27]:
sentiList_cv

[('suck', -0.79368416404904929),
 ('cannot', -0.73521816628823489),
 ('well', 1.4833375586097639),
 ('tin', -0.74060142893083669),
 ('nice', 1.024431009610399),
 ('cancel', -0.64193097779377994),
 ('cheap', -1.2248568218564291),
 ('tri', -0.96080516679581673),
 ('noth', -0.54817526703463992),
 ('speak', -1.2361787898561589),
 ('scare', -0.51521103027314064),
 ('learn', 0.55557991341972524),
 ('answer', -0.59954451127046171),
 ('enjoy', 1.3602428101131103),
 ('alway', -0.94001293247103779),
 ('back', -1.0712698130171625),
 ('annoy', 0.55728474382488014),
 ('right', 0.55769586562848095),
 ('thank', 0.65036722656779866),
 ('road', -0.68591407451541064),
 ('weather', 0.68008757758466565),
 ('hue', -0.55375636177067822),
 ('bose', -0.57093402835370222),
 ('screen', -0.50517478191641485),
 ('excel', 1.0091464789867797),
 ('tooth', 0.52431279349104953),
 ('unplug', -0.97759733361128409),
 ('hardli', -1.0650081427506832),
 ('took', 0.66755784911233784),
 ('spanish', -0.80213617491717826),
 ('d

In [28]:
# Contextual Sentiments for TfIdf Vectors
sentiList_tf = []
threshold = 0.5
for word, index in vocab_tf.items():
    weight = tf_coeffs[0][tf_feat_dict[word]]
    if weight > threshold or weight < -threshold:
        sentiList_tf.append((word, weight))

In [29]:
len(sentiList_tf)

166

In [30]:
sentiList_tf

[('suck', -0.87075754610411826),
 ('cannot', -0.78528422777689921),
 ('well', 1.1431913659154069),
 ('nice', 1.0315367982227033),
 ('wish', 0.649775371495665),
 ('everytim', -0.61973439756016568),
 ('cheap', -1.2944361090543091),
 ('tri', -1.8048567080658127),
 ('home', 0.57528988612136212),
 ('noth', -0.75702757574167567),
 ('disappoint', -0.53684386250724803),
 ('speak', -1.10543362189422),
 ('learn', 1.0282080246513838),
 ('smart', 0.58323325918794822),
 ('answer', -0.6663176240082318),
 ('enjoy', 1.2035594815599644),
 ('alway', -0.75364175436722658),
 ('back', -1.5977392942050153),
 ('hous', 0.7496623707849025),
 ('googl', -0.53680725013642516),
 ('thank', 0.69151167858512186),
 ('everyth', 0.86520925496932044),
 ('weather', 0.7591504839136558),
 ('hue', -0.64118860908756159),
 ('bose', -0.50776021421906925),
 ('screen', -1.0838977423113909),
 ('excel', 0.80939803830246659),
 ('wast', -0.58952554747076436),
 ('unplug', -0.95804724076149139),
 ('said', -0.59792952729532489),
 ('hard

In [31]:
[word[0] for word in sentiList_tf]

['suck',
 'cannot',
 'well',
 'nice',
 'wish',
 'everytim',
 'cheap',
 'tri',
 'home',
 'noth',
 'disappoint',
 'speak',
 'learn',
 'smart',
 'answer',
 'enjoy',
 'alway',
 'back',
 'hous',
 'googl',
 'thank',
 'everyth',
 'weather',
 'hue',
 'bose',
 'screen',
 'excel',
 'wast',
 'unplug',
 'said',
 'hardli',
 'spanish',
 'daughter',
 'cool',
 'massiv',
 'emoji',
 'reboot',
 'card',
 'connect',
 'yet',
 'awesom',
 'love',
 'switch',
 'dot',
 'littl',
 'send',
 'everi',
 'randomli',
 'live',
 'repair',
 'unrespons',
 'reconnect',
 'unnecessari',
 'spoken',
 'password',
 'buy',
 'nois',
 'like',
 'fan',
 'wonder',
 'cant',
 'perfectli',
 'none',
 'disconnect',
 'enter',
 'reason',
 'respond',
 'use',
 'month',
 'upset',
 'batteri',
 'fix',
 'listen',
 'question',
 'goe',
 'new',
 'control',
 'came',
 'scroll',
 'youtub',
 'good',
 'echo',
 'spi',
 'refurbish',
 'time',
 'alexi',
 'forc',
 'want',
 'much',
 'amazon',
 'twice',
 'trust',
 'origin',
 'dumb',
 'pair',
 'half',
 'would',
 's

In [32]:
polarized_dict = get_feat_dict(list(set([word[0] for word in sentiList_tf]).intersection(set([word[0] for word in sentiList_cv]))))

In [33]:
# Creating the Bag of Words model with Count Vectors on polarized words
cv_polarized = CountVectorizer(vocabulary = polarized_dict)

In [34]:
X = cv_polarized.fit_transform(corpus).toarray()
y = data.iloc[:, 4].values

In [35]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [36]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
lr_cv_polarized = LogisticRegression()
lr_cv_polarized.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# Predicting the Test set results
y_pred_cv_polarized = lr_cv_polarized.predict(X_test)

In [38]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score,roc_curve
cm2 = confusion_matrix(y_test, y_pred_cv_polarized)
ac2 = accuracy_score(y_test, y_pred_cv_polarized)

In [39]:
print('Confusion Matrix for LR: \n',cm)
print('\n')
print('Accuracy for LR: \n',ac2)

Confusion Matrix for LR: 
 [[ 12  42]
 [  0 576]]


Accuracy for LR: 
 0.934920634921


In [40]:
#XGBoost for Contextual Sentiment
#Using XGBoost
import xgboost as xgb

from xgboost import XGBClassifier
classifier2 = XGBClassifier()
classifier2.fit(X_train,y_train)
y_pred = classifier2.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('\n')
print('Accuracy for XGBoost: \n',accuracy_score(y_test, classifier2.predict(X_test)))

[[ 10  44]
 [  0 576]]


Accuracy for XGBoost: 
 0.930158730159


## Embeddings 

In [41]:
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
# Others
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup as soup
from nltk.stem.snowball import SnowballStemmer

In [42]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.layers import Flatten, LSTM, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Dropout, Activation, Input, Dense, concatenate
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

Using TensorFlow backend.


In [43]:
revdata = pd.read_csv('amazon_alexa.tsv', delimiter = '\t', 
                      quoting = 3)

In [44]:
revdata.shape

(3150, 5)

In [45]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [46]:
#Applying the above function to column verified reviews
revdata['verified_reviews'] = revdata['verified_reviews'].map(lambda x : clean_text(x))

In [47]:
revdata.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,love echo !,1
1,5,31-Jul-18,Charcoal Fabric,love it !,1
2,4,31-Jul-18,Walnut Finish,sometim play game answer question correct alex...,1
3,5,31-Jul-18,Charcoal Fabric,lot fun thing old learn dinosaur control light...,1
4,5,31-Jul-18,Charcoal Fabric,music,1


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(revdata['verified_reviews'].values, revdata['feedback'], test_size = 0.25, random_state = 101)

In [49]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocabulary_size = len(tokenizer.word_index) + 1
vocabulary_size

2577

In [50]:
print(X_train[0])

my new best friend ! ! i am littl behind time i have never anyth like this anoth alexa googl home super amaz use tool use play sleep sound sleep use amazon audibl book set alarm remind receiv notif play game her tell short stori lol pretti fun know peopl complain speaker i am sure whi i am sure open concept hous want blast music might blast around loud enough rent room bedroom perfect problem speaker love everyth it !


In [51]:
trainseq = tokenizer.texts_to_sequences(X_train)
print(len(trainseq))

2362


In [52]:
trainseq = pad_sequences(trainseq,maxlen=100,padding='post')

In [53]:
testseq = tokenizer.texts_to_sequences(X_test)
testseq = pad_sequences(testseq,padding='post',maxlen=100)

In [54]:
print(trainseq.shape)
print(testseq.shape)

(2362, 100)
(788, 100)


In [55]:
emb_size = 16

In [56]:
op_units = revdata['feedback'].nunique

In [64]:
#Define the model
model = Sequential()
model.add(Embedding(vocabulary_size,emb_size,input_length=100))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [71]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 16)           41232     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                16010     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 57,253
Trainable params: 57,253
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
# fit the model
model.fit(trainseq, y_train, epochs=5, verbose=2)

Epoch 1/5
 - 4s - loss: 0.3471 - acc: 0.9005
Epoch 2/5
 - 1s - loss: 0.2677 - acc: 0.9149
Epoch 3/5
 - 1s - loss: 0.2388 - acc: 0.9183
Epoch 4/5
 - 1s - loss: 0.1799 - acc: 0.9352
Epoch 5/5
 - 1s - loss: 0.1160 - acc: 0.9610


In [74]:
# evaluate the model
loss, accuracy = model.evaluate(testseq, y_test, verbose=2)
print('Loss: %f' % (loss))
print('Accuracy: %f' % (accuracy*100))

Loss: 0.162841
Accuracy: 93.781726


## Pretrained Embeddings

### Gensim word2vec

In [75]:
review_corpus = revdata['verified_reviews'].values.tolist()

In [76]:
len(review_corpus)

3150

In [79]:
review_corpus = []
for i in range(len(revdata['verified_reviews'])):
    review = revdata['verified_reviews'][i].lower()
    review = review.split()
    review_corpus.append(review)

In [80]:
len(review_corpus)

3150

In [82]:
review_corpus[0]

['love', 'echo', '!']

In [83]:
# Word2Vec Data processing
from gensim.models import Word2Vec
model = Word2Vec(sentences=review_corpus, size=100, 
                window=5, workers=4, min_count=100)

C:\Users\hi\Anaconda3\envs\tfdeeplearning\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [84]:
words = list(model.wv.vocab)
print("Vocabulary size: %d"% len(words))

Vocabulary size: 89


In [85]:
filename = 'review_w2v.txt'
model.wv.save_word2vec_format(filename, binary=False)

### Using pretrained w2v as embeddings

In [86]:
import os

embeddings_index_w2v = {}
f = open(os.path.join('','review_w2v.txt'), encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coeffs = np.asarray(values[1:])
    embeddings_index_w2v[word] = coeffs
f.close()

In [87]:
# Vectorize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review_corpus)
seqs = tokenizer.texts_to_sequences(review_corpus)

In [88]:
# Pad sequences
word_index = tokenizer.word_index
print("Found %d unique tokens"% len(word_index))

review_pad = pad_sequences(seqs, maxlen = 50)
review_pad.shape

Found 2944 unique tokens


(3150, 50)

In [89]:
num_words = len(word_index) + 1
embedding_matrix_w2v = np.zeros((num_words, 100))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index_w2v.get(word)
    if embedding_vector is not None:
        # words not found in embeddings_index_w2v will be all zeros
        embedding_matrix_w2v[i] = embedding_vector
print(num_words)    

2945


In [94]:
# define the model
model = Sequential()
embedding_layer = Embedding(num_words, 
                            100,
                            embeddings_initializer = Constant(embedding_matrix_w2v),
                            input_length=100,
                            trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [95]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [96]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 100)          294500    
_________________________________________________________________
flatten_7 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                100010    
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 394,521
Trainable params: 100,021
Non-trainable params: 294,500
_________________________________________________________________
None


In [98]:
# fit the model
model.fit(trainseq, y_train, epochs=10, verbose=2)

Epoch 1/10
 - 1s - loss: 0.2509 - acc: 0.9229
Epoch 2/10
 - 1s - loss: 0.2444 - acc: 0.9234
Epoch 3/10
 - 1s - loss: 0.2382 - acc: 0.9242
Epoch 4/10
 - 1s - loss: 0.2332 - acc: 0.9255
Epoch 5/10
 - 1s - loss: 0.2290 - acc: 0.9255
Epoch 6/10
 - 1s - loss: 0.2242 - acc: 0.9263
Epoch 7/10
 - 1s - loss: 0.2206 - acc: 0.9280
Epoch 8/10
 - 1s - loss: 0.2161 - acc: 0.9297
Epoch 9/10
 - 1s - loss: 0.2146 - acc: 0.9289
Epoch 10/10
 - 1s - loss: 0.2086 - acc: 0.9335


In [99]:
# evaluate the model
loss, accuracy = model.evaluate(testseq, y_test, verbose=2)
print('Loss: %f' % (loss))
print('Accuracy: %f' % (accuracy*100))

Loss: 0.258423
Accuracy: 93.020305
